# 多层感知机（MLP）

In [32]:
#minist 用MLP实现，MLP也是使用pytorch实现的
import torchvision
import torch
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.optim as optim
import time
from  torch.utils  import data 
from d2l import torch as d2l

## 数据集
    使用Fashion-MNIST图像分类数据集

In [33]:
'''下载数据集'''
# 通过ToTensor实例将图像数据从PIL类型变换成32位浮点数格式，
# 并除以255使得所有像素的数值均在0到1之间
trans = transforms.ToTensor()
#下载训练数据
mnist_train = torchvision.datasets.FashionMNIST(
    root="datasets",  #保存的目录
    train=True,       #下载的是训练数据集
    transform=trans,   #得到的是pytorch的tensor，而不是图片
    download=True)  #从网上下载
#下载测试数据
mnist_test = torchvision.datasets.FashionMNIST(
    root="datasets", train=False, transform=trans, download=True)
len(mnist_train),len(mnist_test)

(60000, 10000)

In [34]:
'''装载数据集'''
data_loader_train=data.DataLoader(dataset=mnist_train,
                                                batch_size=64,
                                                shuffle=True)   #数据是否打乱
data_loader_test=data.DataLoader(dataset=mnist_test,
                                    batch_size=64,
                                    shuffle=True)

## 定义模型

In [35]:
num_inputs,num_hidden,num_outputs=28*28,256,10
lr=0.1
num_epochs=10

In [36]:
'''定义了一个隐藏层'''
net=torch.nn.Sequential(
    torch.nn.Flatten(),   #将数据战平
    torch.nn.Linear(num_inputs,num_hidden),  #隐藏层
    torch.nn.ReLU(),
    torch.nn.Linear(num_hidden,num_outputs)  #输出层
)
'''定义权重'''
def  init_w(m):
    if type(m)==torch.nn.Linear:
        torch.nn.init.normal_(m.weight,std=0.01)
net.apply(init_w) 

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=10, bias=True)
)

In [37]:
'''相关优化器'''
loss=torch.nn.CrossEntropyLoss()  #交叉熵
optimizer=torch.optim.SGD(net.parameters(),lr=lr)

## 训练

In [38]:
'''简单训练'''
d2l.train_ch3(net,data_loader_train,data_loader_test,loss,num_epochs,optimizer)

'简单训练'

In [39]:
d2l.predict_ch3(net,data_loader_test)

In [42]:
'''定义预测准确率函数'''
def acc(y_hat,y):
    '''
    :param y_hat: 接收二维张量，例如 torch.tensor([[1], [0]...])
    :param y: 接收二维张量，例如 torch.tensor([[0.1, 0.2, 0.7], [0.8, 0.1, 0.1]...]) 三分类问题
    :return:
    '''
    y_hat=y_hat.argmax(axis=1)
    cmp=y_hat.type(y.dtype)==y  #数据类型是否相同
    return float(cmp.type(y.dtype).sum())
    
class Accumulator():
    ''' 对评估的正确数量和总数进行累加 '''
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, item):
        return self.data[item]

'''自定义每个批次训练函数'''
def train_epoch_cha3(net,data_loader_train,loss,optimizer):
    #判断是不是pytorch得model，如果是，就打开训练模式，pytorch得训练模式默认开启梯度更新
    if isinstance(net,torch.nn.Module):
        net.train()
    #创建样本累加器【累加每批次的损失值、样本预测正确的个数、样本总数】
    metric = Accumulator(3)  
    for x,y in data_loader_train:
        #前向传播获取预测结果
        y_hat=net(x)
        #计算损失
        l=loss(y_hat,y) 
        #判断是pytorch自带得方法还是我们手写得方法（根据不同得方法有不同得处理方式）
        if isinstance(optimizer,torch.optim.Optimizer):
            #梯度清零
            optimizer.zero_grad()
            #损失之求和，反向传播（pytorch自动进行了损失值计算）
            l.backward()
            #更新梯度
            optimizer.step()
            #累加个参数
            metric.add(
                float(l)*len(y),  #损失值总数
                acc(y_hat,y),     #计算预测正确得总数
                y.size().numel()  #样本总数
            )
    #返回平均损失值，预测正确得概率
    return metric[0]/metric[2],metric[1]/metric[2]

'''正式训练'''
def train_cha3(num_epochs,net,data_loader_train,loss,optimizer):
    for epoch in range(num_epochs):
        #返回平均损失值和正确率
        train_metrics=train_epoch_cha3(net,data_loader_train,loss,optimizer)
        print(f"epoch{epoch+1}:loss={train_metrics[0]},acc={train_metrics[1]*100:.2f}%")

In [43]:
train_cha3(num_epochs,net,data_loader_train,loss,optimizer)

epoch1:loss=0.3681945420026779,acc=86.71%
epoch2:loss=0.34939849710464477,acc=87.28%
epoch3:loss=0.334820201532046,acc=87.88%
epoch4:loss=0.31987783874670667,acc=88.43%
epoch5:loss=0.3096299751520157,acc=88.81%
epoch6:loss=0.30028981035550434,acc=89.05%
epoch7:loss=0.291978483804067,acc=89.32%
epoch8:loss=0.2833652744293213,acc=89.64%
epoch9:loss=0.2762533922433853,acc=89.80%
epoch10:loss=0.2684988775253296,acc=90.15%


## 模型测试

In [44]:
'''测试模型'''
def test_cha3(net,test_iter):
    if isinstance(net,torch.nn.Module):
        net.eval()  #将模型设置为评估模式
    metric=Accumulator(2)
    for x,y in test_iter:
        metric.add(
            acc(net(x),y),  #计算准确个数
            y.numel()  #测试样本总数
        )
    #返回模型得准确率
    print(f"test_acc={metric[0]/metric[1]:.2f}%")
    return metric[0]/metric[1]

'''测试'''
test_cha3(net,data_loader_test)

test_acc=0.88%


0.8753